In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357281 sha256=a9ffd53ebff2ab5de96e80862488033c92a89c3240181573e61925b3fbfacff9
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

Завантажте датасет для рецензій (ml-100k):

In [3]:


data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


Виведіть перші 5 рядків:

In [4]:
df = data.build_full_trainset().build_anti_testset()
print("Перші 5 рецензій:")
for i in range(5):
    print(df[i])

Перші 5 рецензій:
('196', '302', 3.52986)
('196', '377', 3.52986)
('196', '51', 3.52986)
('196', '346', 3.52986)
('196', '474', 3.52986)


Реалізуйте два алгоритми для рекомендаційної системи:

Використайте крос-валідацію для підбору параметрів:

In [5]:
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

Оберіть найкращий алгоритм:

In [6]:

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7247777968639855
Найкращий MAE для KNNBasic: 0.8028233604405386
Вибраний алгоритм: SVD


Вивести рекомендації для конкретного користувача:

In [7]:
best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")


Кількість рецензій користувача 196: 34
Топ-10 фільмів, рекомендованих для користувача:
Фільм 318 з прогнозованим рейтингом 4.57
Фільм 197 з прогнозованим рейтингом 4.57
Фільм 1449 з прогнозованим рейтингом 4.55
Фільм 483 з прогнозованим рейтингом 4.50
Фільм 357 з прогнозованим рейтингом 4.46
Фільм 408 з прогнозованим рейтингом 4.44
Фільм 480 з прогнозованим рейтингом 4.43
Фільм 178 з прогнозованим рейтингом 4.40
Фільм 64 з прогнозованим рейтингом 4.39
Фільм 192 з прогнозованим рейтингом 4.39


# Завдання 2

In [14]:
!pip install scikit-surprise

In [15]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
file_path = '/content/drive/My Drive/user_rating_history.csv'
data = pd.read_csv(file_path)

print(data.columns)
print(data.info())

data.dropna(inplace=True)

reader = Reader(rating_scale=(data['rating'].min(), data['rating'].max()))
dataset = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(dataset, test_size=0.2)

model = SVD()
model.fit(trainset)

predictions = model.test(testset)

rmse = accuracy.rmse(predictions)

print(f"RMSE: {rmse}")

user_id = 1
item_ids = data['movieId'].unique()
user_predictions = [(item_id, model.predict(user_id, item_id).est) for item_id in item_ids]
user_predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ 5 рекомендацій для юзера:", user_id)
for item_id, predicted_rating in user_predictions[:5]:
    print(f"Item ID: {item_id}, Predicted Rating: {predicted_rating:.2f}")

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None
RMSE: 0.8677
RMSE: 0.8676846214706453
Топ 5 рекомендацій для юзера: 1
Item ID: 260, Predicted Rating: 5.00
Item ID: 527, Predicted Rating: 5.00
Item ID: 1954, Predicted Rating: 5.00
Item ID: 2329, Predicted Rating: 5.00
Item ID: 2959, Predicted Rating: 5.00
